In [48]:
import pandas as pd
pd.options.display.max_columns = None
import re
import numpy as np

Seguiremos trabajando con mismos csv que en el pair anterior, el de los empleados que unimos en el pair de unión de datasets y el de los tickets de todos los años. Para ello, lo primero que tendremos que hacer es cargar estos dos csv.

1. Cread una columna nueva y una función que nos de el mes en el que se solicitó el servicio. Tened en cuenta que todas las filas tienen la misma estructura. A la hora de escribir la función debemos tener en cuenta que puede que no haya la información de mes en la columna de la fecha. En ese caso devolved un nulo (NaN).

📌 Nota La fecha tendrá que ir formato de string como "Enero", "Febrero", etc.

💡 Pista 💡Podréis usar regex, pero no es estrictamente necesario.

In [49]:
df_pe = pd.read_csv("datos/mergeado.csv") #
df_pe.head(1)

,Unnamed: 0,ID Puesto,Puesto,Departamento,Tipo Puesto,Turno,ID Empleado,Apellido,Nombre,Email
0,0,10-23200,Ingeniero de Materiales,MATERIALES,Operativo,NaN,725,Villa,Eliana,eliana .villa@tractchun.com


In [50]:
df_copia_pe= df_pe.copy()
df_copia_pe.head(1)

,Unnamed: 0,ID Puesto,Puesto,Departamento,Tipo Puesto,Turno,ID Empleado,Apellido,Nombre,Email
0,0,10-23200,Ingeniero de Materiales,MATERIALES,Operativo,NaN,725,Villa,Eliana,eliana .villa@tractchun.com


In [51]:
df_tickets = pd.read_csv("datos/concatenado.csv")
df_tickets.head(1)

,Unnamed: 0,ID Ticket,Fecha,ID Empleado,ID Agente,Categoría,Tipo,Severidad,Prioridad,Días Resolución,Satisfacción,año
0,0,GDDENR-5042564453,2016-07-13,1735.0,4.0,Acceso&%Login,Problema,0 - Sin Clasificar,0 - Sin Asignar,0.0,5.0,2016


In [52]:
df_copia_ticket= df_tickets.copy()
df_copia_ticket.head(1)

,Unnamed: 0,ID Ticket,Fecha,ID Empleado,ID Agente,Categoría,Tipo,Severidad,Prioridad,Días Resolución,Satisfacción,año
0,0,GDDENR-5042564453,2016-07-13,1735.0,4.0,Acceso&%Login,Problema,0 - Sin Clasificar,0 - Sin Asignar,0.0,5.0,2016


In [53]:
df_copia_ticket['Fecha'].isnull().sum() #comprobamos que hay 0 nulos

0

In [54]:
df_copia_ticket['Fecha']= pd.to_datetime(df_copia_ticket['Fecha']) #pasamos a fecha el formato str

In [55]:
def sacar_mes(x): #función para extraer el mes 
        try:
                return x.strftime("%B")
        except:
            return np.nan
                

In [56]:
df_copia_ticket["mes"] = df_copia_ticket["Fecha"].apply(sacar_mes)
df_copia_ticket.head(1)

,Unnamed: 0,ID Ticket,Fecha,ID Empleado,ID Agente,Categoría,Tipo,Severidad,Prioridad,Días Resolución,Satisfacción,año,mes
0,0,GDDENR-5042564453,2016-07-13,1735.0,4.0,Acceso&%Login,Problema,0 - Sin Clasificar,0 - Sin Asignar,0.0,5.0,2016,July


2. En la tabla de empleados, tenemos los nombres y los apellidos de los empleados, pero en columnas separadas. En los ejercicios de pair de la lección del groupby vimos que si quisieramos agrupar por los nombres, esto se nos puede complicar. Ahora nos piden que creemos una columna donde tengamos el nombre y el apellido juntos.

📌 NOTA La columna nueva debe tener el nombre y apellido con la primera letra en mayúsculas, por ejemplo "Jesús Villa".

📌 NOTA Una vez hallamos creado la columna nueva, eliminad las columnas de nombre y apellido originales.

In [57]:
df_copia_pe.head(1)

,Unnamed: 0,ID Puesto,Puesto,Departamento,Tipo Puesto,Turno,ID Empleado,Apellido,Nombre,Email
0,0,10-23200,Ingeniero de Materiales,MATERIALES,Operativo,NaN,725,Villa,Eliana,eliana .villa@tractchun.com


In [58]:
def sacar_nombre(nombre, apellido):
    return nombre + ' ' + apellido


In [59]:
df_nombre = df_copia_pe.apply(lambda dataframe: sacar_nombre(dataframe['Nombre'], dataframe['Apellido']), axis=1 )
df_nombre_completo =pd.DataFrame(df_nombre)

In [60]:
df_nombre_completo.rename(columns={0: 'Nombre_Apellidos'}, inplace=True)
df_nombre_completo

,Nombre_Apellidos
0,Eliana Villa
1,FEDERICO CHAVEZ
2,ANGEL FIGUEROA
3,Gonzalo Piedrahita
4,Gonzalo González
...,...
1995,GAMBOA FIGUEROA
1996,LOURDES MATA
1997,ELENA MORALES
1998,DIOGENES MEDINA


In [ ]:
mergeado_izq = pd.merge(left=nombre, right=df_species, how='left',
                        left_on='species_id', right_on='species_id')


3. En el pair programming desde a dirección nos pedían que agruparamos los datos en base a la columna de "categoría", puede que en aquel momento nos dieramos cuenta que había varías categorías que eran igual (en concreto "Acceso Login") pero que en algunos casos tenía unos símbolos raros (&, %, -). Es el momento de limpiar esta columna. El objetivo, reemplazar todos estos símbolos raros por "/", para conseguir homogeneizar los valores de esta columna. Para ellos deberemos crear una función.

4. Una vez que hayáis terminado eliminad las columnas originales conservando las nuevas.

5. Guardad el dataframe en un csv que usaremos más adelante